<a href="https://colab.research.google.com/github/cimbelli/Inland_Navigable_Waterway/blob/main/s1_danube_1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Derive the total km of Serbian navigable waterways of rivers and canals in year ...

In [11]:
year = '2019'

In [12]:
### comment below after the first run

#!pip install geojson
#!pip install earthengine-api

#url = 'https://raw.githubusercontent.com/cimbelli/Inland_Navigable_Waterway/main/waterway_poly6.geojson'
#!wget $url
#f = open('waterway_poly6.geojson')
#data1 = json.load(f)

In [13]:
import ee
#import time
import matplotlib
import numpy as np
import requests
#import geemap
#from urllib.request import urlopen
#import urllib

import os
import time
import geojson
import datetime
import json
import csv

import folium
#import geopandas as gpd

In [14]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
           
def download(url, tipo, name):
    response = requests.get(url)
    
    if tipo == "raster":
        imgref = os.path.join(name + ".tif")
        with open(imgref, 'wb') as fd:
            fd.write(response.content)
    else:
        vec = os.path.join(name + ".geojson")
        with open(vec, 'wb') as fd:
            fd.write(response.content)
            
    return None

def download_raster(img, name):
    url = img.getDownloadUrl({
        'bands': ['VV'],
        'region': bb,
        'scale' : 10,
        'format': 'GEO_TIFF',
        'maxPixels': 834607840,
        'formatOptions': {
            'cloudOptimized': True
          }
    })
    download(url,"raster", name)
    return None

In [15]:
#start_time = time.time()
# Trigger the authentication flow.

#ee.Authenticate()



# Initialize the library.
ee.Initialize()
maxPixels: 6362493000

sDate0 = year + '-01-01'    #last 6 months 2021
fDate = year + '-12-31'

csvfilename = "Serbia_navigable_inland_waterways_" + year + ".csv"

image_viz_params = {
    'bands': ['VV'],
    'min': 0.0,
    'max': 1.0,
    #'gamma': [0.95, 1.1, 1]
}

In [ ]:
header = ['id', 'length', 'vessels']
with open(csvfilename, 'w', encoding='UTF8') as f1:
    writer = csv.writer(f1)
    writer.writerow(header)
    

    tot_len_rivers = 0
    tot_len_canals = 0
    for i in range(len(data1['features'])):
        #print(data1['features'][i]['properties']['id'])
        
        coords = data1['features'][i]['geometry']['coordinates']
        [j.pop(2) for j in coords[0][0]]
        id1 = data1['features'][i]['properties']['id']
        type1 = data1['features'][i]['properties']['type']
        lenkm1 = data1['features'][i]['properties']['len_km']
        aoi = ee.Geometry.Polygon(coords[0][0])
        
        #if id1 > 20:
            #print(id1)

        imgVV = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')   #S1_GRD_FLOAT
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) 
                .filter(ee.Filter.eq('instrumentMode', 'IW'))
                .select('VV')
                .filterDate(sDate0, fDate)
                .filterBounds(aoi)
                #.reduce(ee.Reducer.percentile([50]))
                )

        imgVV_fixed = imgVV.max().clip(aoi).toUint8()
        im = imgVV_fixed

        center = list(aoi.centroid().getInfo().values())[1][1], list(aoi.centroid().getInfo().values())[1][0]

        ####### map
        #map1 = folium.Map(location=center, zoom_start=10)
        #folium.Map.add_ee_layer = add_ee_layer
        #map1.add_ee_layer(im, image_viz_params, 'false color composite')
        #display(map1)

        classes = im.reduceToVectors(
            **{
            'reducer': ee.Reducer.countEvery(),
            'geometry': aoi.getInfo(),
            'scale': 10,
            'maxPixels': 100000000
        })

        vessels = (ee.FeatureCollection(classes).filter(ee.Filter.gt('count', 15)).filter(ee.Filter.lt('count', 10000)))
        vessels_centroids = vessels.map(lambda f: f.centroid(10))

        #vessels_centroids1 = vessels_centroids.intersection(aoi)
        vessels_centroids1 = vessels_centroids.filter(ee.Filter.bounds(aoi))
        n_vessels = vessels_centroids1.size().getInfo()


        print("there are", n_vessels, "vessels", "in the", type1, "with id", id1, "and lenght", lenkm1, "km", "n_img", imgVV.size().getInfo())
        writer.writerow([id1, lenkm1, n_vessels])
        
        if n_vessels > 0:
            #downloadUrl1 = vessels_centroids1.getDownloadURL(**{'filetype': 'GEOJSON'})
            #download(downloadUrl1, "vessels", "vessels_id_" + str(id1) + "_" + year)
            if type1 == "river":
                tot_len_rivers += lenkm1
            else:
                tot_len_canals += lenkm1

print()
print(tot_len_rivers, "kms of Navigable rivers and lakes")
print(tot_len_canals, "kms of Navigable canals")
print("Total of", tot_len_rivers + tot_len_canals, "kms of Inland waterways")

--2022-09-27 13:56:41--  https://raw.githubusercontent.com/cimbelli/Inland_Navigable_Waterway/main/waterway_poly6.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5680561 (5.4M) [text/plain]
Saving to: ‘waterway_poly6.geojson.2’

waterway_poly6.geoj 100%[===================>]   5.42M  --.-KB/s    in 0.1s    

2022-09-27 13:56:42 (56.0 MB/s) - ‘waterway_poly6.geojson.2’ saved [5680561/5680561]

there are 9 vessels in the river with id 1 and lenght 143 km n_img 240
there are 1 vessels in the river with id 2 and lenght 92 km n_img 238
there are 1 vessels in the river with id 3 and lenght 54 km n_img 296
there are 2 vessels in the river with id 4 and lenght 88 km n_img 180
there are 5 vessels in the river with id 5 and lenght 45 km n_img 179
there are 0 v